In [56]:
import json
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [57]:
file_path = "/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/nonca_fullserv_rnd3.csv"
ca_ff = pd.read_csv(file_path)
file_path_2 = "/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/uszips.csv"
ca_zip_count = pd.read_csv(file_path_2)
file_path_3 = "/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/processed_prices_ubereats_ca_ff_03222024.csv"
example = pd.read_csv(file_path_3)

In [58]:
#Drop all the columns we don't need 
ca_ff_ = ca_ff.drop(columns=['Unnamed: 0', 'inputted_location','restaurant_distance'])
ca_ff_ = ca_ff_.dropna(subset=['restaurant_location'])

In [59]:
ca_zip_count = ca_zip_count[['zip', 'county_name']]

In [60]:
#restaurant_rating cleaning 
# Ensure the column is of string type using .loc
ca_ff_.loc[:, 'restaurant_rating'] = ca_ff_['restaurant_rating'].astype(str)

# Count rows containing 'mi'
rows_with_mi = ca_ff_['restaurant_rating'].str.contains('mi').sum()
print("Number of rows with 'mi' in restaurant rating:", rows_with_mi)

# Replace invalid ratings ending with 'mi' with '0' using .loc
ca_ff_.loc[:, 'restaurant_rating'] = ca_ff_['restaurant_rating'].str.replace(r'.*mi$', '0', regex=True)

Number of rows with 'mi' in restaurant rating: 0


In [61]:
#converting data types 
ca_ff_['restaurant_name'] = ca_ff_['restaurant_name'].astype('string')
ca_ff_['menu_item'] = ca_ff_['menu_item'].astype('string')
ca_ff_['menu_item'] = ca_ff_['menu_item'].str.replace(r'\s+', ' ', regex=True)
ca_ff_['restaurant_location'] = ca_ff_['restaurant_location'].astype('string')
ca_ff_['restaurant_rating'] = ca_ff_['restaurant_rating'].str.strip().astype(float)

In [62]:
#cleaning up string columns 

ca_ff_['menu_item'] = ca_ff_['menu_item'].str.lower()
ca_ff_['restaurant_location'] = ca_ff_['restaurant_location'].str.lower()
ca_ff_['restaurant_name'] = ca_ff_['restaurant_name'].str.replace('_', ' ')

#remove special characters
ca_ff_['menu_item'] = ca_ff_['menu_item'].apply(lambda x: ''.join(ch for ch in x if ch.isalnum() or ch.isspace()))
ca_ff_

,Unnamed: 0.1,restaurant_name,menu_item,menu_item_price,restaurant_location,restaurant_rating,number_of_ratings
0,0,Chilis,triple dipper,18.59,"4600 chapel hill blvd., durham, nc, 27707-2669...",4.2,700+
1,1,Chilis,southwestern eggrolls,14.59,"4600 chapel hill blvd., durham, nc, 27707-2669...",4.2,700+
2,2,Chilis,skillet queso,12.09,"4600 chapel hill blvd., durham, nc, 27707-2669...",4.2,700+
3,3,Chilis,boneless wings,0.00,"4600 chapel hill blvd., durham, nc, 27707-2669...",4.2,700+
4,4,Chilis,white skillet queso,9.99,"4600 chapel hill blvd., durham, nc, 27707-2669...",4.2,700+
...,...,...,...,...,...,...,...
137774,132728,Red Robin,coleslaw,4.79,"95 n moorland rd, brookfield, wi, 53005, us",4.4,470+
137775,132729,Red Robin,garlic fries,4.79,"95 n moorland rd, brookfield, wi, 53005, us",4.4,470+
137776,132730,Red Robin,yukon chips,3.59,"95 n moorland rd, brookfield, wi, 53005, us",4.4,470+
137777,132731,Red Robin,garlic parmesan broccoli,4.79,"95 n moorland rd, brookfield, wi, 53005, us",4.4,470+


In [63]:
def price_list(x):
    return list(x)

def mean_non_zero(x):
    return np.mean(x[x != 0]) if np.any(x != 0) else 0

def median_non_zero(x):
    return np.median(x[x != 0]) if np.any(x != 0) else 0

def std_non_zero(x):
    return np.std(x[x != 0]) if np.any(x != 0) else 0

In [64]:
#Outback Steakhouse 

ca_ff_outback = ca_ff_[ca_ff_['restaurant_name'] == 'Outback Steakhouse']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_outback = ca_ff_outback.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_outback.columns = [' '.join(col).strip() for col in grouped_outback.columns.values]



In [65]:
#Second part of grouping 
outback_lst = ['aussie fries','boomerang cheeseburger','the outbacker burger', ]

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_outback = ca_ff_outback[ca_ff_outback['menu_item'].isin(outback_lst)].sort_values('menu_item')
menu_items_outback = menu_items_outback.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_outback_2 = menu_items_outback.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_outback_2[['fries', 'cheeseburger', 'hamburger']] = grouped_outback_2['menu_item_price'].apply(pd.Series)
grouped_outback_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_outback = pd.merge(grouped_outback, grouped_outback_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_outback['combo'] = np.nan
merged_outback['specialty_item'] = np.nan
merged_outback

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,fries,cheeseburger,hamburger,combo,specialty_item
0,Outback Steakhouse,"100 buckwalter parkway, bluffton, sc, 29910, us",19.080200,14.990,13.627205,4.2,102,1,4.99,10.49,14.99,NaN,NaN
1,Outback Steakhouse,"100 n pointe blvd, lancaster, pa, 17601, us",16.931310,12.790,12.506728,4.2,86,800+,4.99,10.49,14.99,NaN,NaN
2,Outback Steakhouse,"10001 westheimer road # 1010, houston, tx, 770...",18.213061,13.585,13.213504,4.6,100,4,4.99,10.49,14.99,NaN,NaN
3,Outback Steakhouse,"10220 north michigan road, carmel, in, 46077, us",17.925189,13.585,13.061469,4.4,109,1,4.99,10.49,13.99,NaN,NaN
4,Outback Steakhouse,"10901 university avenue, clive, ia, 50325, us",18.074242,13.180,13.527068,4.4,101,1,4.99,10.49,14.99,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,Outback Steakhouse,"9498 brownsboro road, louisville, ky, 40241, us",17.816355,13.990,12.638780,4.5,110,1,4.99,10.49,13.99,NaN,NaN
144,Outback Steakhouse,"9501 quivira drive, overland park, ks, 66214, us",15.150787,10.990,11.671453,4.5,99,2,4.39,9.39,14.28,NaN,NaN
145,Outback Steakhouse,"9753 fm 1960 bypass, humble, tx, 77338, us",17.834000,13.990,12.716984,4.4,108,2,4.99,10.49,13.99,NaN,NaN
146,Outback Steakhouse,"missingaddress, madison, wi, 53717, us",19.014688,15.490,13.497096,4.5,98,700+,4.99,10.49,14.99,NaN,NaN


In [66]:
#Red Robin

ca_ff_rr = ca_ff_[ca_ff_['restaurant_name'] == 'Red Robin']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_rr = ca_ff_rr.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_rr.columns = [' '.join(col).strip() for col in grouped_rr.columns.values]

In [67]:
#Second part of grouping 
rr_lst = ['reds cheeseburger','steak fries', ]

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_rr = ca_ff_rr[ca_ff_rr['menu_item'].isin(rr_lst)].sort_values('menu_item')
menu_items_rr = menu_items_rr.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_rr_2 = menu_items_rr.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_rr_2[['cheeseburger', 'fries']] = grouped_rr_2['menu_item_price'].apply(pd.Series)
grouped_rr_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_rr = pd.merge(grouped_rr, grouped_rr_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_rr['combo'] = np.nan
merged_rr['specialty_item'] = np.nan
merged_rr['hamburger'] = np.nan
merged_rr

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,cheeseburger,fries,combo,specialty_item,hamburger
0,Red Robin,"10 cabela drive, garner, nc, 27529, us",12.180678,11.490,8.283934,4.5,119,1,8.39,4.79,NaN,NaN,NaN
1,Red Robin,"100, columbia, sc, 29212, us",12.107797,10.990,8.303988,4.3,238,460+,8.39,4.79,NaN,NaN,NaN
2,Red Robin,"1000 greengate centre blvd, greensburg, pa, 15...",12.165630,11.190,8.238932,4.6,120,360+,8.39,4.79,NaN,NaN,NaN
3,Red Robin,"101 east brannon rd, nicholasville, ky, 40356, us",11.762072,10.590,8.400570,4.5,112,390+,8.39,4.79,NaN,NaN,NaN
4,Red Robin,"10211, collierville, tn, 38017, us",12.301475,11.440,8.277475,4.6,123,1,8.39,4.79,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,Red Robin,"95 n moorland rd, brookfield, wi, 53005, us",12.087297,10.590,8.541836,4.4,224,470+,8.39,4.79,NaN,NaN,NaN
82,Red Robin,"9810 riverside pkwy, tulsa, ok, 74137, us",11.814545,10.090,8.401863,4.4,111,210+,8.39,4.79,NaN,NaN,NaN
83,Red Robin,"9870 von allmen court, louisville, ky, 40241, us",12.254220,10.790,8.610464,4.6,110,460+,8.39,4.79,NaN,NaN,NaN
84,Red Robin,"9990 e 13th st, wichita, ks, 67206, us",10.602449,11.175,5.450333,4.3,98,1,7.11,3.60,NaN,NaN,NaN


In [68]:
#Panera

ca_ff_panera = ca_ff_[ca_ff_['restaurant_name'] == 'Panera Bread']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_panera = ca_ff_panera.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_panera.columns = [' '.join(col).strip() for col in grouped_panera.columns.values]

In [69]:
grouped_panera['cheeseburger'] = np.nan
grouped_panera['hamburger'] = np.nan
grouped_panera['specialty_item'] = np.nan
grouped_panera['combo'] = np.nan
grouped_panera['fries'] = np.nan

In [70]:
grouped_panera

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,cheeseburger,hamburger,specialty_item,combo,fries
0,Panera Bread,"103 west farm to market 544, suite 101, murphy...",8.190840,7.89,5.797355,4.5,378,250+,NaN,NaN,NaN,NaN,NaN
1,Panera Bread,"11244 tara blvd, suite 100, hampton, ga, 30228...",8.783960,7.59,8.352094,4.5,155,130+,NaN,NaN,NaN,NaN,NaN
2,Panera Bread,"120 outfield drive, madison, al, 35758, us",8.775185,7.89,6.875157,4.8,142,85,NaN,NaN,NaN,NaN,NaN
3,Panera Bread,"1410 ne douglas street, lee's summit, mo, 6408...",8.726620,8.39,6.285961,4.7,148,31,NaN,NaN,NaN,NaN,NaN
4,Panera Bread,"16209 west 87th street parkway, lenexa, ks, 66...",9.271818,8.39,7.710221,4.4,150,100+,NaN,NaN,NaN,NaN,NaN
5,Panera Bread,"1968 peachtree rd nw, atlanta, ga, 30309, us",7.894598,6.89,6.021397,4.5,91,500+,NaN,NaN,NaN,NaN,NaN
6,Panera Bread,"204 shoemaker road, pottstown, pa, 19464, us",9.171119,7.79,8.597047,4.6,150,26,NaN,NaN,NaN,NaN,NaN
7,Panera Bread,"2135 state street, new albany, in, 47150, us",9.226283,7.79,9.726508,4.7,120,72,NaN,NaN,NaN,NaN,NaN
8,Panera Bread,"2156 north morton street, franklin, in, 46131, us",9.237826,7.79,8.802143,4.9,145,25,NaN,NaN,NaN,NaN,NaN
9,Panera Bread,"277 harbison blvd, columbia, sc, 29212, us",8.813256,8.69,6.288016,4.9,272,10,NaN,NaN,NaN,NaN,NaN


In [71]:
#Denny

ca_ff_denny = ca_ff_[ca_ff_['restaurant_name'] == 'Denny']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_denny = ca_ff_denny.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_denny.columns = [' '.join(col).strip() for col in grouped_denny.columns.values]

In [72]:
#Second part of grouping 
denny_lst = ['seasoned fries','single cheeseburger']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_denny = ca_ff_denny[ca_ff_denny['menu_item'].isin(denny_lst)].sort_values('menu_item')
menu_items_denny = menu_items_denny.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_denny_2 = menu_items_denny.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_denny_2[['fries', 'cheeseburger']] = grouped_denny_2['menu_item_price'].apply(pd.Series)
grouped_denny_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_denny = pd.merge(grouped_denny, grouped_denny_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_denny['combo'] = np.nan
merged_denny['specialty_item'] = np.nan
merged_denny['hamburger'] = np.nan

merged_denny

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,fries,cheeseburger,combo,specialty_item,hamburger
0,Denny,"1-35 north parmer lane, austin, tx, 78753, us",9.201340,8.410,5.252395,4.4,312,600+,4.66,12.78,NaN,NaN,NaN
1,Denny,"101 n. 1200th east, lehi, ut, 84043, us",10.439709,9.030,6.144765,4.0,111,290+,5.39,13.61,NaN,NaN,NaN
2,Denny,"1015 spur 350 west, euless, tx, 76040, us",11.386392,9.550,6.884608,3.9,105,500+,5.81,16.52,NaN,NaN,NaN
3,Denny,"1020 tanger dr., locust grove, ga, 30248, us",10.726381,9.030,6.488356,4.0,113,260+,5.81,14.34,NaN,NaN,NaN
4,Denny,"10433 n. central exprwy, dallas, tx, 75230, us",11.370381,9.340,6.862513,4.4,226,600+,5.81,16.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,Denny,"9567 dyer st, el paso, tx, 79924, us",9.830000,8.990,5.937237,4.5,113,1,4.99,13.09,NaN,NaN,NaN
173,Denny,"9650 s 20th st, oak creek, wi, 53154, us",10.203218,7.470,6.537663,4.3,186,350+,5.81,14.02,NaN,NaN,NaN
174,Denny,"975 s main st, kernersville, nc, 27284, us",10.255000,8.565,6.376734,4.3,222,330+,5.29,13.09,NaN,NaN,NaN
175,Denny,"9766 katy fwy ste b, houston, tx, 77055, us",9.152105,7.390,5.733414,4.6,103,900+,4.59,12.69,NaN,NaN,NaN


In [73]:
#Applebee

ca_ff_apple = ca_ff_[ca_ff_['restaurant_name'] == 'Applebee']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_apple = ca_ff_apple.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_apple.columns = [' '.join(col).strip() for col in grouped_apple.columns.values]

In [74]:
#Second part of grouping 
apple_lst = ['classic burger', 'classic cheeseburger']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_apple = ca_ff_apple[ca_ff_apple['menu_item'].isin(apple_lst)].sort_values('menu_item')
menu_items_apple = menu_items_apple.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_apple_2 = menu_items_apple.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_apple_2[['hamburger', 'cheeseburger']] = grouped_apple_2['menu_item_price'].apply(pd.Series)
grouped_apple_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_apple = pd.merge(grouped_apple, grouped_apple_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_apple['combo'] = np.nan
merged_apple['specialty_item'] = np.nan
merged_apple['fries'] = np.nan


merged_apple

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,hamburger,cheeseburger,combo,specialty_item,fries
0,Applebee,"1000 west esplanada avenue, kenner, la, 70065, us",20.166068,14.49,22.519718,4.5,240,1,12.99,13.49,NaN,NaN,NaN
1,Applebee,"10006 will way, louisville, ky, 40291, us",19.686552,14.19,21.633554,4.6,119,170+,12.39,12.99,NaN,NaN,NaN
2,Applebee,"1001 e. first street, ankeny, ia, 50021, us",22.601653,15.00,25.747257,4.6,124,460+,14.50,15.00,NaN,NaN,NaN
3,Applebee,"101 northern way, york, pa, 17402, us",18.865926,13.89,17.865360,4.0,111,600+,12.19,13.39,NaN,NaN,NaN
4,Applebee,"1020 e state hwy 152, mustang, ok, 73064, us",20.555138,12.99,23.721555,4.4,112,200+,12.39,12.99,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,Applebee,"995 i-35, new braunfels, tx, 78130, us",20.540000,16.29,19.221724,4.5,105,700+,13.79,14.99,NaN,NaN,NaN
214,Applebee,"996 e. connelly court, park city, ks, 67219, us",22.393802,15.00,25.809027,4.1,124,160+,13.25,14.50,NaN,NaN,NaN
215,Applebee,"greentree mall, clarksville, in, 47129, us",21.347522,14.19,23.581581,4.1,116,330+,12.39,12.99,NaN,NaN,NaN
216,Applebee,"indian brook dr, dover, nh, 03820, us",24.908898,16.29,26.091151,4.3,128,700+,14.49,15.79,NaN,NaN,NaN


In [75]:
#TGI Fridays

ca_ff_tgi = ca_ff_[ca_ff_['restaurant_name'] == 'TGI Fridays']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_tgi = ca_ff_tgi.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_tgi.columns = [' '.join(col).strip() for col in grouped_tgi.columns.values]

In [76]:
#Second part of grouping 
tgi_lst = ['cheeseburger', 'seasoned fries']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_tgi = ca_ff_tgi[ca_ff_tgi['menu_item'].isin(tgi_lst)].sort_values('menu_item')
menu_items_tgi = menu_items_tgi.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_tgi_2 = menu_items_tgi.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_tgi_2[['cheeseburger', 'fries']] = grouped_tgi_2['menu_item_price'].apply(pd.Series)
grouped_tgi_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_tgi = pd.merge(grouped_tgi, grouped_tgi_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_tgi['combo'] = np.nan
merged_tgi['specialty_item'] = np.nan
merged_tgi['hamburger'] = np.nan
merged_tgi

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,cheeseburger,fries,combo,specialty_item,hamburger
0,TGI Fridays,"1100 timber dr e, garner, nc, 27529, us",25.258202,16.660,26.478770,4.400000,554,1,13.56,3.21,NaN,NaN,NaN
1,TGI Fridays,"1114 woodruff road, greenville, sc, 29607, us",25.430000,16.430,26.936822,4.300000,554,900+,13.33,3.09,NaN,NaN,NaN
2,TGI Fridays,"115 hendersonvile rd, asheville, nc, 28803, us",25.214552,16.660,26.444838,4.200000,417,1,13.56,3.21,NaN,NaN,NaN
3,TGI Fridays,"12811 s tryon st, charlotte, nc, us, us",25.212015,16.660,26.445256,3.866427,417,1,13.56,3.21,NaN,NaN,NaN
4,TGI Fridays,"1516 s willow st, manchester, nh, 03103, us",25.022797,16.430,26.425060,3.966348,419,1,13.33,3.09,NaN,NaN,NaN
5,TGI Fridays,"15190 crossroads pkwy, gulfport, ms, 39503, us",23.596985,15.170,26.132995,4.233568,426,1,12.87,2.86,NaN,NaN,NaN
6,TGI Fridays,"1835 n highway 17, mount pleasant, sc, 29464, us",24.305882,15.740,26.465922,4.400000,426,2,13.56,3.21,NaN,NaN,NaN
7,TGI Fridays,"20430 highway 59 n, humble, tx, us, us",23.667528,15.400,26.155161,4.300000,566,1,12.98,2.86,NaN,NaN,NaN
8,TGI Fridays,"220 coneflower drive, garland, tx, 75040, us",23.629657,15.400,26.120513,4.432864,426,1,12.98,2.86,NaN,NaN,NaN
9,TGI Fridays,"240 mall blvd, monroeville, pa, 15146, us",25.212015,16.660,26.445256,4.200000,417,1,13.56,3.21,NaN,NaN,NaN


In [77]:
#'Buffalo Wild Wings'
#TGI Fridays

ca_ff_buff = ca_ff_[ca_ff_['restaurant_name'] == 'Buffalo Wild Wings']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_buff = ca_ff_buff.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_buff.columns = [' '.join(col).strip() for col in grouped_buff.columns.values]

In [78]:
#Second part of grouping 
buff_lst = ['allamerican cheeseburger', 'regular french fries']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_buff = ca_ff_buff[ca_ff_buff['menu_item'].isin(buff_lst)].sort_values('menu_item')
menu_items_buff = menu_items_buff.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_buff_2 = menu_items_buff.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_buff_2[['cheeseburger', 'fries']] = grouped_buff_2['menu_item_price'].apply(pd.Series)
grouped_buff_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_buff = pd.merge(grouped_buff, grouped_buff_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_buff['combo'] = np.nan
merged_buff['specialty_item'] = np.nan
merged_buff['hamburger'] = np.nan
merged_buff

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,cheeseburger,fries,combo,specialty_item,hamburger
0,Buffalo Wild Wings,"1000 bower pkwy, columbia, sc, 29212, us",7.509082,5.64,8.861520,4.1,107,900+,13.29,4.79,NaN,NaN,NaN
1,Buffalo Wild Wings,"1007 w university ave ste 1, georgetown, tx, 7...",7.770404,6.99,8.829073,4.2,108,2,13.29,4.79,NaN,NaN,NaN
2,Buffalo Wild Wings,"1018 crossings blvd, spring hill, tn, 37174, us",7.842708,6.74,9.044434,4.2,105,320+,13.79,4.79,NaN,NaN,NaN
3,Buffalo Wild Wings,"103 quarry rd, downingtown, pa, 19335, us",8.157525,6.99,9.257959,4.3,110,600+,15.29,5.29,NaN,NaN,NaN
4,Buffalo Wild Wings,"10497 gateway blvd w, el paso, tx, 79925, us",7.720396,6.49,8.871802,4.2,110,2,13.79,4.79,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,Buffalo Wild Wings,"9263 n., owasso, ok, 74055, us",8.062929,6.99,8.996233,4.4,107,220+,14.49,5.29,NaN,NaN,NaN
176,Buffalo Wild Wings,"9402 northlake west dr, charlotte, nc, 28216, us",7.863960,6.99,8.945037,4.0,110,1,13.99,4.79,NaN,NaN,NaN
177,Buffalo Wild Wings,"9701 roosevelt blvd, philadelphia, pa, 19114, us",8.160200,7.14,9.283081,4.1,108,1,14.79,4.79,NaN,NaN,NaN
178,Buffalo Wild Wings,"985 e fort union blvd, midvale, ut, 84047, us",7.842813,6.49,9.092927,4.1,105,1,13.79,4.79,NaN,NaN,NaN


In [79]:
#Stack all restaurants
ubereats_fullserv_prices = pd.concat([merged_outback, grouped_panera, merged_rr, merged_buff, merged_tgi, merged_apple, merged_denny]).reset_index(drop=True)

In [84]:
#If there are bad addresses, replace them with the actual address here 

ubereats_fullserv_prices.loc[434, 'restaurant_location'] = "12811 s tryon st, charlotte, nc, 28273"
ubereats_fullserv_prices.loc[438, 'restaurant_location'] = "20430 us-59, humble, tx, 77338"
ubereats_fullserv_prices.loc[701, 'restaurant_location'] = "13635 san pedro ave, san antonio, tx, 78232"
ubereats_fullserv_prices.loc[742, 'restaurant_location'] = "2500 sm 291 hwy, independence, mo, 64055"
ubereats_fullserv_prices.loc[843, 'restaurant_location'] = "9001 shawnee mission pkwy, merriam, ks, 66202"

In [83]:
nan_zip_rows = ubereats_fullserv_prices[ubereats_fullserv_prices['zip'].isna()]
nan_zip_rows

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,fries,cheeseburger,hamburger,combo,specialty_item,state,zip
434,TGI Fridays,"12811 s tryon st, charlotte, nc, us, us",25.212015,16.66,26.445256,3.866427,417,1,3.21,13.56,NaN,NaN,NaN,None,None
438,TGI Fridays,"20430 highway 59 n, humble, tx, us, us",23.667528,15.40,26.155161,4.300000,566,1,2.86,12.98,NaN,NaN,NaN,None,None
701,Denny,"13635 san pedro ave, san antonio, tx, us, us",9.597619,8.41,5.886331,4.200000,113,800+,4.77,13.19,NaN,NaN,NaN,None,None
742,Denny,"2500 sm 291 hwy, independence, mo, us, us",9.558286,8.41,5.859777,4.200000,113,270+,4.77,13.19,NaN,NaN,NaN,None,None
843,Denny,"9001 shawnee mission parkway, merriam, ks, us, us",9.639808,8.41,5.892176,4.000000,112,360+,4.77,12.98,NaN,NaN,NaN,None,None


In [85]:
pattern = r",\s*([a-zA-Z]{2})\s*,?\s*(\d{5}(?:-\d{4})?)"

def extract_state_zip(address):
    match = re.search(pattern, address)
    if match:
        state, zip_code = match.groups()
        return state, zip_code
    else:
        return None, None

# Apply the function to extract state and zip code
ubereats_fullserv_prices[['state', 'zip']] = ubereats_fullserv_prices['restaurant_location'].apply(lambda x: pd.Series(extract_state_zip(x)))
ubereats_fullserv_prices['zip'] = ubereats_fullserv_prices['zip'].str.split('-').str[0].astype(int)

#Get county 
ubereats_fullserv_prices = ubereats_fullserv_prices.merge(ca_zip_count, on = 'zip')

In [86]:
specific_date = datetime.strptime('05162024', '%m%d%Y')
# Assign the datetime object to the entire 'date' column
ubereats_fullserv_prices['date'] = specific_date
ubereats_fullserv_prices['uber_eats'] = 1
ubereats_fullserv_prices['post_policy'] = 1
ubereats_fullserv_prices['fast_food'] = 0
ubereats_fullserv_prices['local'] = 0 

In [87]:
#Save as csv 
ubereats_fullserv_prices.to_csv('final_processed_prices_ubereats_nonca_fullserv_05162024.csv', index = True)